# Guided Project: Analyzing CIA Factbook Data Using SQL

In [1]:
# Activating ipython-sql
!conda install -yc conda-forge ipython-sql

/bin/sh: 1: conda: not found


## Introduction

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/), a compendium of statistics about all of the countries on Earth.

First step is to connect to the database:

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

## Overview of the Data

In [3]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Query that returns the first five rows of the facts table in the database:

In [4]:
%%sql
SELECT *
    FROM facts
    LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Summary Statistics

Writing a single query that returns the following:

- Minimum population
- Maximum population
- Minimum population growth
- Maximum population growth

In [5]:
%%sql
SELECT  MIN(population) AS minimum_pop,
        MAX(population) AS maximum_pop,
        MIN(population_growth) AS min_pop_growth,
        MAX(population_growth) AS max_pop_growth
    FROM facts;

Done.


minimum_pop,maximum_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


## Exploring Outliers

Writing a query that returns the countries with the minimum population.

In [6]:
%%sql
SELECT *
    FROM facts
    WHERE population = (SELECT MIN(population)
    FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


Writing a query that returns the countries with the maximum population.

In [7]:
%%sql
SELECT *
    FROM facts
    WHERE population = (SELECT MAX(population)
    FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


## Exploring Average Population and Area

Recomputing the summary statistics found earlier while excluding the row for the whole world. IncludING the following:

- Minimum population
- Maximum population
- Minimum population growth
- Maximum population growth

In [8]:
%%sql
SELECT  MIN(population) AS minimum_pop,
        MAX(population) AS maximum_pop,
        MIN(population_growth) AS min_pop_growth,
        MAX(population_growth) AS max_pop_growth
    FROM facts
    WHERE name <> "World";

Done.


minimum_pop,maximum_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


Calculating the average value for the following columns:
- Population
- Area

In [9]:
%%sql
SELECT  AVG(population) AS avg_population,
        AVG(area) AS avg_area
    FROM facts
    WHERE name <> 'World';

Done.


avg_population,avg_area
32242666.56846473,555093.546184739


## Finding Densely Populated Countries

Writing a query that finds all countries meeting both of the following criteria:
- The population is above average.
- The area is below average.

In [10]:
%%sql
SELECT *
    FROM facts
    WHERE population > (SELECT AVG(population)
                            FROM facts  WHERE name <> 'World'
                    )

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
7,ar,Argentina,2780400,2736690,43710,43431886,0.93,16.64,7.33,0.0
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
24,br,Brazil,8515770,8358140,157630,204259812,0.77,14.46,6.58,0.14
28,bm,Burma,676578,653508,23070,56320206,1.01,18.39,7.96,0.28
32,ca,Canada,9984670,9093507,891163,35099836,0.75,10.28,8.42,5.66
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44
38,co,Colombia,1138910,1038700,100210,46736728,1.04,16.47,5.4,0.64
40,cg,"Congo, Democratic Republic of the",2344858,2267048,77810,79375136,2.45,34.88,10.07,0.27
